In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials 

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1ZCiTN-crxvIzWCFmzDFy_y4qQcf58lP-' # url에서 file id만 copy/paste
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('titanic_train.csv')

In [0]:
import pandas as pd
titanic_df = pd.read_csv('titanic_train.csv')

In [14]:
# Aggregation 

## NaN 값은 제외
print(titanic_df.count())
print('########################')
print(titanic_df[['Age', 'Fare']])
print('########################')
print(titanic_df[['Age', 'Fare']].mean()) # 평균
print('########################')
print(titanic_df[['Age', 'Fare']].sum())
print('########################')
print(titanic_df[['Age', 'Fare']].count())
print('########################')

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64
########################
      Age     Fare
0    22.0   7.2500
1    38.0  71.2833
2    26.0   7.9250
3    35.0  53.1000
4    35.0   8.0500
..    ...      ...
886  27.0  13.0000
887  19.0  30.0000
888   NaN  23.4500
889  26.0  30.0000
890  32.0   7.7500

[891 rows x 2 columns]
########################
Age     29.699118
Fare    32.204208
dtype: float64
########################
Age     21205.1700
Fare    28693.9493
dtype: float64
########################
Age     714
Fare    891
dtype: int64
########################
<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [16]:
titanic_groupby = titanic_df.groupby(by='Pclass')
print(type(titanic_groupby))
print(titanic_groupby)

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [27]:
# DataFrameGroupBy 객체에 Aggregation 함수 호출 하여 Group by 수행.
titanic_groupby = titanic_df.groupby('Pclass').count()
titanic_groupby

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,216,216,216,216,186,216,216,216,216,176,214
2,184,184,184,184,173,184,184,184,184,16,184
3,491,491,491,491,355,491,491,491,491,12,491


In [29]:
print(type(titanic_groupby))
print(titanic_groupby.shape)
print(titanic_groupby.index)

titanic_groupby = titanic_df.groupby(by='Pclass')[['PassengerId', 'Survived']].count()
print(titanic_groupby)
print('########################')
print(titanic_df[['Pclass', 'PassengerId', 'Survived']].groupby('Pclass').count()) # groupby 'Pclass'는 []안에 포함되어야함.
print('########################')
print(titanic_df.groupby('Pclass')['Pclass'].count()) # 특정 컬럼의 값만 보고 싶을때
print(titanic_df['Pclass'].value_counts()) # 특정 컬럼의 값만 보고 싶을때(Series만 가능)

<class 'pandas.core.frame.DataFrame'>
(3, 2)
Int64Index([1, 2, 3], dtype='int64', name='Pclass')
        PassengerId  Survived
Pclass                       
1               216       216
2               184       184
3               491       491
########################
        PassengerId  Survived
Pclass                       
1               216       216
2               184       184
3               491       491
########################
Pclass
1    216
2    184
3    491
Name: Pclass, dtype: int64
3    491
1    216
2    184
Name: Pclass, dtype: int64


In [30]:
# max(), min() 두 함수의 값을 모두 알고 싶을때
titanic_df.groupby('Pclass')['Age'].agg([max, min]) 

,max,min
Pclass,,
1,80.0,0.92
2,70.0,0.67
3,74.0,0.42


In [31]:
# dictionary을 통해 여러 aggregation 적용 가능
agg_format = {'Age':'max', 'SibSp':'sum', 'Fare':'mean'} 
titanic_df.groupby('Pclass').agg(agg_format)

,Age,SibSp,Fare
Pclass,,,
1,80.0,90,84.154687
2,70.0,74,20.662183
3,74.0,302,13.675550


In [32]:
# missing data 처리
# isna() : 모든 컬럼들이 NaN 인지 True/False 값 반환
titanic_df.isna().head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False


In [33]:
# 반환 결과에 sum() 호출하여 컬럼별로 NaN 건수 구하기.
titanic_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [41]:
# fillna()로 missing data 대체하기
titanic_df['Cabin'] = titanic_df['Cabin'].fillna('0000')
titanic_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0000,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0000,S


In [40]:
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean())
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')
titanic_df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [45]:
# apply lambda
titanic_df['Name_len'] = titanic_df['Name'].apply(lambda x : len(x))
titanic_df[['Name', 'Name_len']].head(3)

,Name,Name_len
0,"Braund, Mr. Owen Harris",23
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",51
2,"Heikkinen, Miss. Laina",22


In [49]:
titanic_df['Child_Adult'] = titanic_df['Age'].apply(lambda x : 'Child' if x <= 27 else 'Adult')
titanic_df[['Age', 'Child_Adult']].head(3)

,Age,Child_Adult
0,22.0,Child
1,38.0,Adult
2,26.0,Child


In [52]:
titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x : 'Child' if x <= 15  
                                                else ('Adult' if x <= 60 else 'Elderly'))
titanic_df['Age_cat'].value_counts()

Adult      786
Child       83
Elderly     22
Name: Age_cat, dtype: int64

In [53]:
def get_category(age):
  cat1 = ''
  if age <= 5: cat1 = 'Baby'
  elif age <= 12: cat1 = 'Child'
  elif age <= 18: cat1 = 'Teenager'
  elif age <= 25: cat1 = 'Student'
  elif age <= 35: cat1 = 'Young Adult'
  elif age <= 60: cat1 = 'Adult'
  else : cat1 = 'Elderly'
  return cat1

titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x : get_category(x))
titanic_df[['Age', 'Age_cat']].head()

,Age,Age_cat
0,22.0,Student
1,38.0,Adult
2,26.0,Young Adult
3,35.0,Young Adult
4,35.0,Young Adult
